Yue You

Collaborator: Rachael, Tommy

# Problem 1
#### Show the CAR condition, $x \rightarrow Pr(O = o | X = x)$ for $x \in \mathcal{C}(o)$ is constant implies $Pr(X=x| O = o) = Pr(X = x | x \in \mathcal{C}(o))$. You may assume all random variables here are discrete for simplicity.


The CAR condition, $\forall x \in \mathcal{C}(o), Pr(O = o | X = x)$ is constant, 

which means the observation $O = o$ is not influenced by values of $X \in \mathcal{C}(o)$ which is taken, only by the fact that X did take a value in $\mathcal{C}(o)$.

want to show: $\forall o,\ x \in \mathcal{C}(o),\ P_r(O = o |\ X = x) = P_r(O = o|\ X \in \mathcal{C}(o))$............................(1)

By conditional independet assumption: 

Given $X \in \mathcal{C}(o)$, the events $X = x$ and $O = o$ are conditionally independent.

(1) is equivalent to $\forall x \in \mathcal{C}(o),\ P_r[O = o|\ X = x\ and\ X \in \mathcal{C}(o)] = P_r[O=o|\ X \in \mathcal{C}(o)]$,

By symmetry we have:

$P_r[X = x|\ O = o\ and\ x \in \mathcal{C}(o)] = P_r[O=o|\ X \in \mathcal{C}(o)]$.

Since $P_r[X = x|\ O = o\ and\ x \in \mathcal{C}(o)]  = P_r[X = x|\ X \in \mathcal{C}(o)]$

$\therefore P_r(X = x|\ O = o) = P_r[X = x|\ X \in \mathcal{C}(o)]$

$\therefore P_r(O = o|\ X \in \mathcal{C}(o)) = P_r(O = o |\ X = x)$

# Problem 2 
#### Let $P_{X,\epsilon}$ be a path through $P_X$, the distribution of the full data, $X$, and having score $S_1(X)$. This then defines a path $P_{P_{X,\epsilon},G}$ through the observed data distribution, $P_{P_XG}$. Show that the scores generated by these paths are $E[S_1(X) | O = o]$.





$\frac{d}{d\epsilon}\frac{log(p_{P_{X,\epsilon}, G})}{p_{P_{X}, G}}|_{\epsilon = 0}$

$=\frac{d}{d\epsilon}log(p_{P_{X,\epsilon}, G})|_{\epsilon = 0}$

$=\int \frac{p(O=o|X=x)S_1(x)p(x)dv(x)}{p_{P_{X}, G}}$

$=\int S_1(x)p(x|O=o)dv(x)$

$= E(S_1(X)|O = o)$

# Problem 3
#### Let $G_{\epsilon}$ be a path through $G$, the distribution of the censoring time, $C$, given $X$, having score $S_2(C,X)$. This then defines a path $P_{P_XG_{\epsilon}}$ through the observed data distribution, $P_{P_XG}$. Show that the scores generated by these paths are $E[S_2(C, X) | O = o]$.

$\frac{d}{d\epsilon}\frac{log(p_{P_X, G_\epsilon})}{p_{P_{X}, G}}|_{\epsilon = 0}$

$=\frac{d}{d\epsilon}log(p_{P_{X}, G_\epsilon})|_{\epsilon = 0}$

$=\int \frac{p(O=o|X=x)S_2(c,x)p(x)dv(x)}{p_{P_{X}, G}}$

$=\int S_2(c,x)p(x|O=o)dv(x)$

$= E(S_2(X)|O = o)$

# Problem 4
#### This problem involves simulating data under a general Cox model. Let’s make the assumption we have a conditional hazard of death at time, $t$, given by $\lambda(t | X) =  \lambda_0(t)exp(f_{\beta}(X))$ where $X$ is a set of covariates and $f_{\beta}$ is a function indexed by $\beta$, say finite dimensional. Assume the baseline hazard is  $\lambda_0(t) = exp(rt)$ for positive $r$. Given $X$, what is the distribution of death times? Prove your answer.


$S(t|\ x) = exp(-\int^t_0 d\Lambda(S|X))$
$=exp(-\int^t_0 exp(rs)exp(f_\beta(X))ds) \sim Uniform[0,1]$

$\therefore t = \frac{log[-r logU exp(-f_\beta(X))+1]}{r}$, where $U \sim Uniform[0,1]$, $r \gt 0$



# Problem 5
#### Complete the first problem from LabCox in the lab section of the files on bCourses.


In [3]:
library(knitr)
library(survival)
library(ggplot2)
library(mvtnorm)
library(doParallel)
library(survminer)

In [10]:
conf_interval_count <- c()
i <- 0
  while( i< 1000){
  n = 1000
  
  # draw the W from standard normals
  W1 = rnorm(n)
  W2 = rnorm(n)
  g0 = function(W1,W2) plogis(.1 + W1*W2)
  A = rbinom(n, 1, g0(W1,W2))
  
  # draw T and C--both generated with random uniforms as perviously described but C could
  # be generated anyway independent of T given W for identifiability purposes
  C = -log(runif(n))/(.01*exp(.3*W1))
  T =  -log(runif(n))/(.02*exp(2*W1^2 - A))
  
  Ttilde = pmin(T, C)
  Delta = C >= T & T<= 100
  
  # Create the survival objec
  S = Surv(time = Ttilde, event = Delta, type = "right")
  data = data.frame(A = A, W1 = W1, W2 = W2)
  coxfit = coxph(S ~ ., data = data)
  truth <- exp(-1)
  
  conf_interval<- as.data.frame(summary(coxfit)[8])
  upper <- conf_interval$conf.int.upper..95[1]
  lower <- conf_interval$conf.int.lower..95[1]
  
  conf_interval_count <- c(conf_interval_count, as.numeric(truth >= lower && truth <= upper))
  i <- i +1
}

#coverage
sum(conf_interval_count) / 1000


[1] 0.001

# Bonus
#### Assume a CAR model for full data consisting of survival time, censoring time, the continuous baseline covariates and randomly assigned treatment indicator. We have observed data $min(T,C),\Delta$ along with the covariates and treatment indicator. Someome receives a data set of 1000 independent subjects drawn from this model from an RCT and runs a Cox Proportional hazards regression with treatment as the only covariate, showing a significantly negative coefficient. Can you convince this person he may be wrong via simulation? Explain how you set up your simulation and turn in your code to show the results.


In [4]:
n <- 1000
conf_interval_count <- c()
i<- 1
while( i< n){
  #simulate the 'truth' to show this person they are wrong
  
  W1 = rnorm(n)
  W2 = rnorm(n)
  A <- rbinom(n, 1, .5)
    
  # draw T and C--both generated with random uniforms as perviously described but C could be generated anyway independent of T given W for identifiability purposes. We're going to make C and T highly dependent on W1, W2, W3, W4, and W5, for model misspecfication later.
  
  C = abs(W1 + W2 - 10*A)
  T = abs(W1)
  
  #C = rweibull(n,2,(-A*abs(W1 + W2) + 120))
  #T = rexp(n,((1/90)))
    
  Ttilde <- pmin(T, C)
  #Note, I changed the Delta here, otherwise everyone would be "alive" so to speak.
  Delta <-  C >= T & T<= 1
    
  # Create the survival objec
  S = Surv(time = Ttilde, event = Delta, type = "right")
  truth <- 1
  
  #This is how we are going to prove this person they are wrong.
  # They only know of the treatment, A, which is from a randomized trial. 
  #The survival object remains the same.
  # This individual refeused to include any covariates.  
  incomplete_data = data.frame(A = A)
  coxfit = coxph(S ~ A, data = incomplete_data)
  
  conf_interval<- as.data.frame(summary(coxfit)[8])
  upper <- conf_interval$conf.int.upper..95[1]
  lower <- conf_interval$conf.int.lower..95[1]
  
  conf_interval_count <- c(conf_interval_count, as.numeric(truth >= lower & truth <= upper))
  i<- i + 1
}

print(paste("Coverage is: ", (sum(conf_interval_count)/n)))


[1] "Coverage is:  0.644"
